# AI Equity Research Report Generator

This notebook generates a comprehensive equity research report by combining data from a company's annual report with live market data from Yahoo Finance, and then using Google's Gemini model for analysis and generation.

## 1. Installation

This cell installs the required Python libraries for the project: the Google AI SDK, yfinance for market data, PyMuPDF for reading PDF files, and python-dotenv for managing local API keys.

In [ ]:
!pip install -q google-generativeai yfinance PyMuPDF python-dotenv

## 2. Configuration

This cell configures the connection to the Gemini API. It securely loads the `GOOGLE_API_KEY` from either Google Colab's Secrets Manager or a local `.env` file, making the notebook portable and safe to share.

In [ ]:
import google.generativeai as genai
import os
from IPython.display import Markdown

# Load API key from Colab secrets or a local .env file.
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except (ImportError, KeyError):
    from dotenv import load_dotenv
    load_dotenv()
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found. Please set it up in Colab Secrets or a local .env file.")

genai.configure(api_key=GOOGLE_API_KEY)

# Configure the model
model = genai.GenerativeModel('gemini-1.5-flash-latest')
print("Gemini model configured.")

## 3. Upload Annual Report

Run the next cell to upload the company's annual report. The script will extract text from the first 50 pages, which typically contain the most critical summaries and management discussions. This page limit is a safeguard against exceeding the free-tier token limits of the API.

In [ ]:
from google.colab import files
import fitz  # PyMuPDF
import io

MAX_PAGES_TO_READ = 50
annual_report_text = ""

print(f"Please upload the Annual Report PDF (will read first {MAX_PAGES_TO_READ} pages)...")
uploaded_files = files.upload()

if uploaded_files:
    file_name = next(iter(uploaded_files))
    pdf_content = uploaded_files[file_name]

    with fitz.open(stream=pdf_content, filetype="pdf") as doc:
        num_pages = min(len(doc), MAX_PAGES_TO_READ)
        for page_num in range(num_pages):
            page = doc.load_page(page_num)
            annual_report_text += page.get_text()
    
    print(f"\nExtracted {len(annual_report_text):,} characters from the first {num_pages} pages of '{file_name}'.")
else:
    print("\nNo file uploaded. Report will be less detailed.")

## 4. Fetch Live Market Data

This cell defines the target company and fetches the latest market data using the `yfinance` library. This includes a business summary and key stock price metrics over the last year. This live data complements the historical data found in the annual report.

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

# --- Define Stock --- 
company_ticker = "RELIANCE"
company_name = "Reliance Industries Limited"
yf_ticker = f"{company_ticker}.NS"

# --- Fetch Live Market Data ---
print(f"Fetching live market data for {company_name}...")
market_data_context = ""
try:
    ticker = yf.Ticker(yf_ticker)
    hist = ticker.history(period="1y")
    
    if hist.empty:
        raise ValueError(f"No price data found for {yf_ticker}.")

    high_52_week = hist['High'].max()
    low_52_week = hist['Low'].min()
    last_close = hist['Close'].iloc[-1]
    yearly_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
    
    info = ticker.info
    business_summary = info.get('longBusinessSummary', 'Not available.')

    market_data_context = f"""
    Business Summary: {business_summary}

    Key Stock Price Metrics:
    - Last Closing Price: ₹{last_close:,.2f}
    - 52-Week High: ₹{high_52_week:,.2f}
    - 52-Week Low: ₹{low_52_week:,.2f}
    - 1-Year Return: {yearly_return:.2f}%
    """
    print("Live market data fetched successfully.")
except Exception as e:
    print(f"Could not fetch market data from yfinance. Error: {e}")
    market_data_context = "Live market data could not be fetched."

## 5. Verify Fetched Data (Optional)

This cell is for debugging. It prints the key variables created in the data-fetching step to verify that the information was retrieved correctly before it is sent to the AI model.

In [ ]:
# This cell is for debugging and verifying the data fetched from yfinance.
print("--- Checking yfinance Outputs ---")

print("\n[Business Summary]:")
print(business_summary)
print("\n" + "="*50)

print("\n[Stock Price Metrics]:")
# Check if the 'last_close' variable exists and is a number before printing
if 'last_close' in locals() and isinstance(last_close, (int, float)):
    print(f"Last Close Price: ₹{last_close:,.2f}")
    print(f"52-Week High: ₹{high_52_week:,.2f}")
    print(f"52-Week Low: ₹{low_52_week:,.2f}")
    print(f"1-Year Return: {yearly_return:.2f}%")
else:
    print("Stock price metrics were not fetched.")

--- Checking yfinance Outputs ---

[Business Summary]:
Reliance Industries Limited engages in the hydrocarbon exploration and production, oil and chemicals, retail, and digital service businesses worldwide. It operates through Oil to Chemicals, Oil and Gas, Retail, Digital Services, and Others segments. The company offers refining and marketing products, including liquefied petroleum gas, propylene, naphtha, gasoline, jet/aviation turbine fuel, kerosine oil, diesel, sulphur, and petroleum coke. It also provides polymers, including high-density and low-density polyethylene (PE), linear low-density PE, homopolymer, random and impact copolymer, and polyvinyl chloride; fibre intermediates, such as purified terephthalic acid, and ethylene glycols and oxide; aromatics, such as paraxylene, ortho xylene, benzene, and linear alkyl benzene and paraffin; and textiles comprising fabrics, apparel, and auto furnishings. In addition, the company offers elastomers, such as polybutadiene rubber, styrene butadiene rubber, and butyl rubber; fibre and yarn polyesters; and bioenergy solutions, consisting of compressed biogas, and pellets and briquettes. Further, it engages in oil and gas exploration and production activities; and operates various stores comprising supermarket, hypermarket, wholesale cash and carry, specialty, and online stores, as well as stores that offer apparel, beauty and cosmetics, accessories, footwear, consumer electronics, and others. Additionally, the company operates media and entertainment platforms, and Network18 and television channels; publishes magazines; and offers highway hospitality and fleet management services. It also provides digital services, including connectivity, fibre, mobile devices, apps, business, and other digital solutions. The company was founded in 1957 and is based in Mumbai, India.

==================================================

[Stock Price Metrics]:
Last Close Price: ₹1,375.00
52-Week High: ₹1,544.83
52-Week Low: ₹1,110.42
1-Year Return: -7.53%

## 5. Define the Master Prompt

This cell constructs the final, comprehensive prompt that will be sent to the Gemini model. It bundles the extracted annual report text and the live market data together and provides a detailed structure for the desired output.

In [ ]:
comprehensive_prompt = f"""
Act as a senior equity research analyst. Write a highly detailed equity research report for {company_name} ({company_ticker}).

Use the following two sources: the company's Annual Report text and live market data.
Synthesize information from BOTH sources to construct your analysis. Be thorough and provide insightful analysis, not just a summary.

---BEGIN ANNUAL REPORT TEXT---
{annual_report_text}
---END ANNUAL REPORT TEXT---

---BEGIN LIVE MARKET DATA---
{market_data_context}
---END LIVE MARKET DATA---

**Equity Research Report: {company_name} ({company_ticker})**

**1. Executive Summary:**
- Provide a concise overview of the business, its strategic direction, key financial highlights, and your overall investment thesis.

**2. Company Profile & Business Segments:**
- Corporate History & Vision: Summarize the company's evolution and strategic vision from the report.
- Business Segment Deep Dive: For each major business segment, analyze its performance, key drivers, and outlook based on the annual report.

**3. Industry Analysis & Competitive Landscape:**
- Analyze the industry structure for the company's key segments, discussing market size, growth drivers, and major competitors.

**4. In-Depth Financial Analysis (Based on Annual Report):**
- Income Statement Analysis: Analyze trends in revenue, EBITDA, and Profit After Tax.
- Balance Sheet Analysis: Examine debt levels (Debt-to-Equity ratio), liquidity (Current Ratio), and asset base.
- Cash Flow Analysis: Discuss cash from operations, investing, and financing activities.

**5. Management Discussion & Corporate Governance:**
- Summarize key points from the "Management Discussion and Analysis" section.
- Briefly comment on the board structure and key management.

**6. Stock Price Performance & Valuation:**
- Analyze the stock's performance using the provided live metrics.
- Discuss common valuation methodologies (e.g., P/E, EV/EBITDA) and compare to competitors if possible.

**7. Investment Risks:**
- Detail the key risks highlighted in the annual report (market, operational, regulatory).

**8. Final Investment Thesis & Recommendation:**
- Bull Case: Primary reasons for an optimistic outlook.
- Bear Case: Primary risks and reasons for caution.
- Conclusion: Conclude with a final, well-reasoned investment outlook.

**9. Disclaimer:**
- Include a standard professional disclaimer.
"""

## 6. Generate the Report

This final cell sends the request to the Gemini model and displays the generated report.

In [ ]:
print(f"Generating comprehensive equity research report for {company_ticker}...")

try:
    response = model.generate_content(comprehensive_prompt)
    display(Markdown(response.text))
except Exception as e:
    print(f"An error occurred during generation: {e}")

Generating comprehensive equity research report for RELIANCE...
Equity Research Report: Reliance Industries Limited (RELIANCE)

Date: October 26, 2023

Analyst: [Your Name], Senior Equity Research Analyst

1. Executive Summary:

Reliance Industries Limited (RIL) is India's largest private sector enterprise, a diversified conglomerate with significant presence in energy (Oil to Chemicals & Oil and Gas), retail, digital services (Jio), and media and entertainment. While FY24-25 showed resilient performance despite global headwinds, with consolidated revenue increasing by 7.1% to ₹10,71,174 crore and EBITDA growing by 2.9% to ₹1,83,422 crore, the stock has underperformed the market over the past year (-7.53% return). RIL’s ambitious Net Carbon Zero target by 2035 and significant investments in new energy present both significant opportunities and considerable execution risks. While the diversification provides a buffer, the O2C segment’s profitability remains vulnerable to global commodity price fluctuations. We currently rate RIL as a Hold, pending clearer visibility on the success of its new energy initiatives and stabilization in the O2C segment.

2. Company Profile & Business Segments:

Corporate History & Vision: Founded in 1957 by Dhirubhai Ambani, RIL has consistently expanded its footprint across diverse sectors, guided by a vision of "Growth is Life" and a commitment to India's development ("What is good for India is good for Reliance"). The company is transitioning into a new-age deep-tech enterprise, investing heavily in R&D across AI, renewable energy, and advanced materials.

Business Segment Deep Dive:

Oil to Chemicals (O2C): This segment, though experiencing an EBITDA decline of 11.9% in FY24-25 due to weaker fuel cracks and lower petrochemical margins, remains a significant revenue and profit contributor. The annual report highlights resilience through disciplined cost management and strong domestic placement. However, global overcapacity and volatile commodity prices pose ongoing challenges.

Oil and Gas: This segment delivered record EBITDA, driven by increased output from KG-D6 and CBM blocks, boosting India's energy independence. Further production augmentation is planned, aligning with India's increasing gas demand. However, the segment's profitability is sensitive to global LNG prices.

Retail: Reliance Retail maintains its position as India's largest retailer, crossing ₹3,30,000 crore in turnover and expanding its store footprint to 19,340. Growth was driven by strong rural demand and expansion in various consumption baskets (fashion, grocery, electronics). The omni-channel strategy and focus on own brands are key strengths.

Digital Services (Jio): Jio remains a key growth engine, with 488 million subscribers and the world's largest data network. The 5G rollout and fixed wireless access (AirFiber) are driving significant growth, although EBITDA margins contracted slightly. The expansion into AI services represents a potential future growth driver.

Media and Entertainment: The merger of Viacom18 and Star India created JioStar, India's largest media platform, achieving significant viewership gains across various genres. The integrated JioHotstar streaming service experienced record viewership. However, content costs and competition remain significant challenges.

3. Industry Analysis & Competitive Landscape:

O2C: The global refining industry faces overcapacity, particularly from China, leading to volatile margins. Key competitors include global majors like Saudi Aramco, Sinopec, and ExxonMobil. In India, RIL’s integrated model offers a cost advantage.

Oil and Gas: The Indian oil and gas sector is characterized by increasing domestic demand and government initiatives to boost production. Competitors include ONGC and other private players.

Retail: The Indian retail market is experiencing rapid growth, driven by rising disposable incomes and increasing urbanization. Major competitors include Tata Group, Aditya Birla Group, and international players like Walmart. RIL’s scale and omni-channel presence provide a strong competitive advantage.

Digital Services: The Indian telecom sector is highly competitive, with major players like Bharti Airtel and Vodafone Idea. Jio’s dominance in data and 5G presents a strong position.

Media and Entertainment: The Indian media landscape is fragmented, with increasing competition from both domestic and international players in the digital space. Disney Star's strategic alliance with RIL strengthens its position in the market.

4. In-Depth Financial Analysis (Based on Annual Report):

Income Statement Analysis: Consolidated revenue growth was healthy (7.1% YoY), driven primarily by Retail and Digital Services. However, the O2C segment's EBITDA decline negatively impacted overall profitability. Net profit growth was modest (2.9% YoY).

Balance Sheet Analysis: RIL maintains a conservative balance sheet with a Debt-to-Equity ratio of 0.41:1 as of March 31, 2025. The company’s investment-grade credit ratings reflect its strong financial position. Liquidity appears healthy, supported by robust internal cash flow generation.

Cash Flow Analysis: The report highlights strong cash profit generation, which adequately covered the significant capital expenditure (₹1,31,107 crore). Investments were focused on new O2C projects, retail expansion, digital infrastructure, and new energy initiatives.

5. Management Discussion & Corporate Governance:

The Management Discussion and Analysis (MDA) emphasizes RIL’s resilient performance amidst global volatility and highlights the strength of its diversified model. The MDA also emphasizes the company’s strategic shift towards a technology-driven, deep-tech enterprise, focusing on new energy and digital services as key growth drivers. The board structure is diverse, with a mix of executive and independent directors, indicating a reasonably robust corporate governance framework.

6. Stock Price Performance & Valuation:

Stock Performance: The stock price closed at ₹1,375.00, significantly below its 52-week high of ₹1,544.83, indicating a 1-year return of -7.53%. The current price is trading near the lower end of its 52-week range.

Valuation: A precise valuation requires detailed projections, but a comparative analysis using P/E and EV/EBITDA multiples relative to peers (Bharti Airtel, Tata Group companies) would be necessary for a more accurate picture. The current valuation seems to reflect investor concerns regarding the O2C segment's profitability and execution risks in the new energy business.

7. Investment Risks:

Commodity Price Volatility: RIL's O2C and Oil and Gas segments are heavily exposed to global commodity price fluctuations, which can significantly impact profitability.

Geopolitical Risks: Global geopolitical instability can affect crude oil prices and energy demand, impacting RIL's O2C and Oil and Gas businesses.

Regulatory Risks: Changes in government regulations, particularly those related to energy transition and environmental protection, could pose challenges to RIL's operations.

Execution Risk (New Energy): The ambitious Net Carbon Zero target by 2035 requires significant capital investment and technological advancements. Failure to execute on this plan effectively could negatively impact investor sentiment and the company's long-term value.

Competition: Intense competition in the retail, telecom, and media and entertainment sectors poses a constant threat.

8. Final Investment Thesis & Recommendation:

Bull Case: Successful execution of RIL's new energy initiatives, stabilization of global commodity prices, and continued strong growth in retail and digital services could drive significant future growth and enhance the company’s valuation. The diversification across multiple sectors serves as a hedge against sector-specific downturns.

Bear Case: Failure to meet the Net Carbon Zero targets, prolonged weakness in the O2C segment due to global overcapacity and intense price competition, and unexpected regulatory hurdles could significantly impact RIL's future performance and share price.

Conclusion: RIL's diversification and strong fundamentals offer some protection, but the current valuation appears to reflect significant investor uncertainty regarding the company’s new energy ambitions and its O2C segment’s outlook. We recommend a Hold rating, pending greater clarity on the execution of its new energy strategy and stabilization in the O2C sector. Further research and more detailed financial modeling are required to provide a definitive buy or sell recommendation.

9. Disclaimer:

This report is for informational purposes only and does not constitute investment advice. The information contained herein is based on publicly available information and the analyst's independent judgment. The analyst may hold a position in the securities discussed in this report. Investing involves risk, and past performance is not indicative of future results. Consult with a financial advisor before making any investment decisions.tolerance. Past performance is not indicative of future results.